In [190]:
import pandas as pd
import pickle
import statsmodels.api as sm

In [8]:
pointmodel = pd.read_pickle("models/margin_predict.pickle")

In [174]:
#Create 'checktest'
#Step1: Import Data from endofseasonratings
#year=2019
check={}
for year in [2018,2019,2020,2021,2022,2023]:
    with open(f'data/endofseasonratings/{year}check.pkl','rb') as f:
            check[year] = pickle.load(f)




In [161]:
#check

In [171]:
#pd.DataFrame.from_dict(check['ypp'],orient='columns').query("current==True").query("week==1")#.query("away_team=='Colorado State'")
#checktest19 = check.copy()


# Setup Data For Points

#Get the Data where 'current == True'


yards = pd.DataFrame(checktest['ypp']).query("current==True").reset_index(drop=True)[['id','season', 'id', 'week', 'neutral_site', 'home_team', 'away_team','home_points','away_points','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
yards.columns = ['id']+[x+"_ypp" for x in yards.columns[1:]]

tds = pd.DataFrame(checktest['TDRatio']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
tds.columns = ['id']+[x+"_TD" for x in tds.columns[1:]]

tos = pd.DataFrame(checktest['TurnoverRatio']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
tos.columns = ['id']+[x+"_TO" for x in tos.columns[1:]]

mcs = pd.DataFrame(checktest['MCRatio']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
mcs.columns = ['id']+[x+"_MC" for x in mcs.columns[1:]]

fcs = pd.DataFrame(checktest['FailedConvRatio']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
fcs.columns = ['id']+[x+"_FC" for x in fcs.columns[1:]]

hvs = pd.DataFrame(checktest['HavocRatio']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
hvs.columns = ['id']+[x+"_HV" for x in hvs.columns[1:]]

passing = pd.DataFrame(checktest['yardsPerPass']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
passing.columns = ['id']+[x+"_PA" for x in passing.columns[1:]]

rushing = pd.DataFrame(checktest['yardsPerRushAttempt']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
rushing.columns = ['id']+[x+"_RU" for x in rushing.columns[1:]]

fga = pd.DataFrame(checktest['FGAttRatio']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
fga.columns = ['id']+[x+"_FGA" for x in fga.columns[1:]]

fg = pd.DataFrame(checktest['FGRatio']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
fg.columns = ['id']+[x+"_FG" for x in fg.columns[1:]]

fd = pd.DataFrame(checktest['FDRatio']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
fd.columns = ['id']+[x+"_FD" for x in fd.columns[1:]]


pointsdf = (
yards.merge(tds,how='left',on='id')
    .merge(tos,how='left',on='id')
    .merge(mcs,how='left',on='id')
    .merge(fcs,how='left',on='id')
    .merge(hvs,how='left',on='id')
    .merge(passing,how='left',on='id')
    .merge(rushing,how='left',on='id')
    .merge(fga,how='left',on='id')
    .merge(fg,how='left',on='id')
    .merge(fd,how='left',on='id')
    
).assign(margin = lambda x: x['home_points_ypp']-x['away_points_ypp']).assign(HFA = lambda x: (x['neutral_site_ypp']!=True).astype(int))



#Duplicate pointsdf
pointsdf_away=pointsdf.copy()
#Flip Home/Away
newcols=[]
for x in list(pointsdf_away.columns):
    if x.find("home")>=0:
        x=x.replace("home","away")
    elif x.find("away")>=0:
        x=x.replace("away","home")
    else:
        x=x
    newcols.append(x)
    
pointsdf_away.columns=newcols

#Flip HFA & Margin
pointsdf_away['HFA']=pointsdf_away['HFA']*-1
pointsdf_away['margin']=pointsdf_away['margin']*-1


pointsdf = pd.concat([pointsdf,pointsdf_away[pointsdf.columns]]).reset_index(drop=True)

In [152]:
#*********HAVING AN ISSUE WITH DUPLICATES*********** HOW TO FIX IT???**********
#yards.merge(tds,how='left',on='id')
#pd.concat([yards,tds],axis=1)
#display(yards.head(2))
#display(tds.head(2))

#yards.head(200).set_index('id').merge(tds.set_index('id'),left_index=True,right_index=True)#.query("home_team_ypp=='Connecticut'").query("away_team_ypp=='UCF'")

#yards.drop_duplicates(subset=['id']).merge(tds.drop_duplicates(subset=['id']),how='left',on='id')


#Create Interaction Columns
homeoff = [c for c in pointsdf.columns if c.find("home_off")>0]
awayoff = [c for c in pointsdf.columns if c.find("away_off")>0]
homedef = [c for c in pointsdf.columns if c.find("home_def")>0]
awaydef = [c for c in pointsdf.columns if c.find("away_def")>0]

morecols = []
for a,b in zip(homeoff,awaydef):
    newstr = a+'-'+b
    morecols.append(newstr)
    pointsdf[newstr]=pointsdf[a]-pointsdf[b]+1500
for a,b in zip(homedef,awayoff):
    newstr = a+'-'+b
    pointsdf[newstr]=pointsdf[a]-pointsdf[b]+1500
    morecols.append(newstr)

In [166]:
#pointsdf

# Function to Setup Points System

In [164]:
def setuppoints(checktest):
    yards = pd.DataFrame(checktest['ypp']).query("current==True").reset_index(drop=True)[['id','season', 'id', 'week', 'neutral_site', 'home_team', 'away_team','home_points','away_points','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
    yards.columns = ['id']+[x+"_ypp" for x in yards.columns[1:]]
    
    tds = pd.DataFrame(checktest['TDRatio']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
    tds.columns = ['id']+[x+"_TD" for x in tds.columns[1:]]
    
    tos = pd.DataFrame(checktest['TurnoverRatio']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
    tos.columns = ['id']+[x+"_TO" for x in tos.columns[1:]]
    
    mcs = pd.DataFrame(checktest['MCRatio']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
    mcs.columns = ['id']+[x+"_MC" for x in mcs.columns[1:]]
    
    fcs = pd.DataFrame(checktest['FailedConvRatio']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
    fcs.columns = ['id']+[x+"_FC" for x in fcs.columns[1:]]
    
    hvs = pd.DataFrame(checktest['HavocRatio']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
    hvs.columns = ['id']+[x+"_HV" for x in hvs.columns[1:]]
    
    passing = pd.DataFrame(checktest['yardsPerPass']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
    passing.columns = ['id']+[x+"_PA" for x in passing.columns[1:]]
    
    rushing = pd.DataFrame(checktest['yardsPerRushAttempt']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
    rushing.columns = ['id']+[x+"_RU" for x in rushing.columns[1:]]
    
    fga = pd.DataFrame(checktest['FGAttRatio']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
    fga.columns = ['id']+[x+"_FGA" for x in fga.columns[1:]]
    
    fg = pd.DataFrame(checktest['FGRatio']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
    fg.columns = ['id']+[x+"_FG" for x in fg.columns[1:]]
    
    fd = pd.DataFrame(checktest['FDRatio']).query("current==True").reset_index(drop=True)[['id','pregame_home_off','pregame_away_def','pregame_away_off','pregame_home_def']]
    fd.columns = ['id']+[x+"_FD" for x in fd.columns[1:]]

    pointsdf = (
    yards.merge(tds,how='left',on='id')
        .merge(tos,how='left',on='id')
        .merge(mcs,how='left',on='id')
        .merge(fcs,how='left',on='id')
        .merge(hvs,how='left',on='id')
        .merge(passing,how='left',on='id')
        .merge(rushing,how='left',on='id')
        .merge(fga,how='left',on='id')
        .merge(fg,how='left',on='id')
        .merge(fd,how='left',on='id')
        
    ).assign(margin = lambda x: x['home_points_ypp']-x['away_points_ypp']).assign(HFA = lambda x: (x['neutral_site_ypp']!=True).astype(int))

    #Duplicate pointsdf
    pointsdf_away=pointsdf.copy()
    #Flip Home/Away
    newcols=[]
    for x in list(pointsdf_away.columns):
        if x.find("home")>=0:
            x=x.replace("home","away")
        elif x.find("away")>=0:
            x=x.replace("away","home")
        else:
            x=x
        newcols.append(x)
        
    pointsdf_away.columns=newcols
    
    #Flip HFA & Margin
    pointsdf_away['HFA']=pointsdf_away['HFA']*-1
    pointsdf_away['margin']=pointsdf_away['margin']*-1
    
    
    pointsdf = pd.concat([pointsdf,pointsdf_away[pointsdf.columns]]).reset_index(drop=True)

    #Create Interaction Columns
    homeoff = [c for c in pointsdf.columns if c.find("home_off")>0]
    awayoff = [c for c in pointsdf.columns if c.find("away_off")>0]
    homedef = [c for c in pointsdf.columns if c.find("home_def")>0]
    awaydef = [c for c in pointsdf.columns if c.find("away_def")>0]
    
    morecols = []
    for a,b in zip(homeoff,awaydef):
        newstr = a+'-'+b
        morecols.append(newstr)
        pointsdf[newstr]=pointsdf[a]-pointsdf[b]+1500
    for a,b in zip(homedef,awayoff):
        newstr = a+'-'+b
        pointsdf[newstr]=pointsdf[a]-pointsdf[b]+1500
        morecols.append(newstr)

    return pointsdf

# Computations

In [185]:
pointsdata = pd.DataFrame()
for year in [2018,2019,2020,2021,2022,2023]:
    pointsdata = pd.concat([pointsdata,setuppoints(check[year])])


In [187]:
use_columns = [
    'pregame_home_off_ypp',
    'pregame_away_def_ypp', 'pregame_away_off_ypp', 'pregame_home_def_ypp',
     'pregame_home_off_TD',
    'pregame_away_def_TD', 'pregame_away_off_TD', 'pregame_home_def_TD',
    'pregame_home_off_TO', 'pregame_away_def_TO', 'pregame_away_off_TO',
    'pregame_home_def_TO', 'pregame_home_off_MC', 'pregame_away_def_MC',
    'pregame_away_off_MC', 'pregame_home_def_MC', 'pregame_home_off_FC',
    'pregame_away_def_FC', 'pregame_away_off_FC', 'pregame_home_def_FC',
    'pregame_home_off_HV',
    'pregame_away_def_HV', 'pregame_away_off_HV', 'pregame_home_def_HV',
    'pregame_home_off_PA', 'pregame_away_def_PA',
       'pregame_away_off_PA', 'pregame_home_def_PA', 'pregame_home_off_RU',
       'pregame_away_def_RU', 'pregame_away_off_RU', 'pregame_home_def_RU',
    'pregame_home_off_FGA','pregame_away_def_FGA', 'pregame_away_off_FGA', 'pregame_home_def_FGA',
    'pregame_home_off_FG','pregame_away_def_FG', 'pregame_away_off_FG', 'pregame_home_def_FG',
    'pregame_home_off_FD','pregame_away_def_FD', 'pregame_away_off_FD', 'pregame_home_def_FD',
     'HFA'
]

more_columns = ['pregame_home_off_ypp-pregame_away_def_ypp',
 'pregame_home_off_TD-pregame_away_def_TD',
 'pregame_home_off_TO-pregame_away_def_TO',
 'pregame_home_off_MC-pregame_away_def_MC',
 'pregame_home_off_FC-pregame_away_def_FC',
 'pregame_home_off_HV-pregame_away_def_HV',
 'pregame_home_off_PA-pregame_away_def_PA',
 'pregame_home_off_RU-pregame_away_def_RU',
 'pregame_home_off_FGA-pregame_away_def_FGA',
 'pregame_home_off_FG-pregame_away_def_FG',
 'pregame_home_off_FD-pregame_away_def_FD',
 'pregame_home_def_ypp-pregame_away_off_ypp',
 'pregame_home_def_TD-pregame_away_off_TD',
 'pregame_home_def_TO-pregame_away_off_TO',
 'pregame_home_def_MC-pregame_away_off_MC',
 'pregame_home_def_FC-pregame_away_off_FC',
 'pregame_home_def_HV-pregame_away_off_HV',
 'pregame_home_def_PA-pregame_away_off_PA',
 'pregame_home_def_RU-pregame_away_off_RU',
 'pregame_home_def_FGA-pregame_away_off_FGA',
 'pregame_home_def_FG-pregame_away_off_FG',
 'pregame_home_def_FD-pregame_away_off_FD']

In [191]:
pointsdata['prediction']=pointmodel.predict(sm.add_constant(pointsdata[use_columns+more_columns]))

In [201]:
pointsdata.assign(miss = lambda x: abs(x['prediction']-x['margin'])).groupby('season_ypp')['miss'].mean()

season_ypp
2018    14.499040
2019    12.712796
2020    13.171381
2021    12.840069
2022    12.306425
2023    12.324199
Name: miss, dtype: float64

In [234]:
pointsdata.query("season_ypp==2023").query("(home_team_ypp=='Oregon')")

,id,season_ypp,id_ypp,week_ypp,neutral_site_ypp,home_team_ypp,away_team_ypp,home_points_ypp,away_points_ypp,pregame_home_off_ypp,...,pregame_home_def_TO-pregame_away_off_TO,pregame_home_def_MC-pregame_away_off_MC,pregame_home_def_FC-pregame_away_off_FC,pregame_home_def_HV-pregame_away_off_HV,pregame_home_def_PA-pregame_away_off_PA,pregame_home_def_RU-pregame_away_off_RU,pregame_home_def_FGA-pregame_away_off_FGA,pregame_home_def_FG-pregame_away_off_FG,pregame_home_def_FD-pregame_away_off_FD,prediction
152,401524008,2023,401524008,3,False,Oregon,Hawai'i,55.0,10.0,1682,...,1685,1662,1645,1637,1649,1653,1643.0,1639.0,1624,26.310607
180,401524015,2023,401524015,4,False,Oregon,Colorado,42.0,6.0,1689,...,1656,1582,1566,1584,1586,1602,1582.0,1580.0,1570,22.639270
390,401524035,2023,401524035,8,False,Oregon,Washington State,38.0,24.0,1713,...,1617,1591,1558,1588,1591,1607,1588.0,1576.0,1582,12.978325
502,401524047,2023,401524047,10,False,Oregon,California,63.0,19.0,1742,...,1664,1606,1592,1589,1615,1575,1614.0,1628.0,1581,17.993949
596,401524053,2023,401524053,11,False,Oregon,USC,36.0,27.0,1751,...,1397,1365,1387,1406,1396,1382,1402.0,1392.0,1372,15.614381
711,401524063,2023,401524063,13,False,Oregon,Oregon State,31.0,7.0,1782,...,1425,1474,1479,1492,1455,1425,1470.0,1446.0,1426,12.353899
812,401524002,2023,401524002,2,False,Oregon,Texas Tech,38.0,30.0,1684,...,1597,1577,1541,1573,1585,1566,1555.0,1556.0,1542,6.097571
990,401524024,2023,401524024,5,False,Oregon,Stanford,42.0,6.0,1691,...,1678,1598,1580,1583,1645,1616,1599.0,1585.0,1602,15.388014
1099,401524034,2023,401524034,7,False,Oregon,Washington,33.0,36.0,1708,...,1355,1332,1296,1314,1347,1341,1343.0,1345.0,1314,-2.351966
1221,401524044,2023,401524044,9,False,Oregon,Utah,35.0,6.0,1732,...,1581,1666,1677,1620,1653,1619,1636.0,1635.0,1640,5.199667


# Try a different model

In [466]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np

In [495]:
#pointsdata.columns

xpred = pointsdata.query("~((season_ypp==2018)&(week_ypp<5))").query("~prediction.isna()")[use_columns+more_columns+['prediction']]
ypred = pointsdata.query("~((season_ypp==2018)&(week_ypp<5))").query("~prediction.isna()")['margin']


X_train,X_test,y_train,y_test=train_test_split(xpred,ypred,test_size=.2)



In [496]:
lin_reg = LinearRegression().fit(X_train.copy().drop(columns=['prediction']),y_train)

In [497]:
#tree_reg = DecisionTreeRegressor(criterion='absolute_error')
#tree_reg.fit(X_train.copy().drop(columns=['prediction']),y_train)

In [498]:
#display(np.mean(abs(tree_reg.predict(X_test.copy().drop(columns=['prediction']))-y_test)))
display(np.mean(abs(lin_reg.predict(X_test.copy().drop(columns=['prediction']))-y_test)))

#Previous Predictions
display(np.mean(abs(X_test['prediction']-y_test)))

12.273175064599483

12.39071238520497

In [535]:
evalyear = 2022

xin = pointsdata.query(f"season_ypp!={evalyear}").query("~((season_ypp==2018)&(week_ypp<5))").query("~prediction.isna()")[use_columns+more_columns+['prediction']]
yin = pointsdata.query(f"season_ypp!={evalyear}").query("~((season_ypp==2018)&(week_ypp<5))").query("~prediction.isna()")['margin']

xout = pointsdata.query(f"season_ypp=={evalyear}").query("~((season_ypp==2018)&(week_ypp<5))").query("~prediction.isna()")[use_columns+more_columns+['prediction']]
yout = pointsdata.query(f"season_ypp=={evalyear}").query("~((season_ypp==2018)&(week_ypp<5))").query("~prediction.isna()")['margin']

In [536]:
annualtrain = LinearRegression().fit(xin.copy().drop(columns=['prediction']),yin)

In [537]:
display(np.mean(abs(annualtrain.predict(xout.copy().drop(columns=['prediction']))-yout)))
display(np.mean(abs(xout['prediction']-yout)))

12.338672969187675

12.306425374538895

In [538]:
display(np.mean((((annualtrain.predict(xout.copy().drop(columns=['prediction']))>0)==(yout>0)).astype(int))))

np.mean(((xout['prediction']>0)==(yout>0)).astype(int))

0.7128851540616247

0.7002801120448179

In [539]:
#Try GradientBoosting
from sklearn.ensemble import GradientBoostingRegressor



In [547]:
gbr = GradientBoostingRegressor(loss='absolute_error', learning_rate=.1)

In [548]:
gbr_reg = gbr.fit(xin.copy().drop(columns=['prediction']),yin)

In [549]:
display(np.mean(abs(gbr.predict(xout.copy().drop(columns=['prediction']))-yout)))

12.454885119592143

# Test Some Stuff

In [310]:
pointsdata['upset']=((pointsdata['margin']>0)!=(pointsdata['prediction']>0)).astype(int)
pointsdata['homeupset']=pointsdata['upset']*(pointsdata['margin']<0).astype(int)
pointsdata['homewin']=(pointsdata['margin']>0).astype(int)



In [317]:
Xu = pointsdata.dropna(subset='prediction').query("(week_ypp>5)&(season_ypp<2023)")[use_columns+more_columns+['prediction']]
Yu = pointsdata.dropna(subset='prediction').query("(week_ypp>5)&(season_ypp<2023)")['homewin']

In [318]:
wintest = sm.Logit(Yu,Xu).fit()

         Current function value: 0.535966
         Iterations: 35


/Users/bennettbernard/opt/anaconda3/envs/fballenv/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [319]:
pointsdata['winprediction']=wintest.predict(pointsdata[use_columns+more_columns+['prediction']])

In [334]:

display(pointsdata.query("season_ypp==2023").query("(home_team_ypp=='Washington')and(away_team_ypp=='Oregon')").iloc[:,3:11])
display(pointsdata.query("season_ypp==2023").query("(home_team_ypp=='Washington')and(away_team_ypp=='Oregon')").iloc[:,11:19])
display(pointsdata.query("season_ypp==2023").query("(home_team_ypp=='Washington')and(away_team_ypp=='Oregon')").iloc[:,19:27])
display(pointsdata.query("season_ypp==2023").query("(home_team_ypp=='Washington')and(away_team_ypp=='Oregon')").iloc[:,27:35])
display(pointsdata.query("season_ypp==2023").query("(home_team_ypp=='Washington')and(away_team_ypp=='Oregon')").iloc[:,35:45])
display(pointsdata.query("season_ypp==2023").query("(home_team_ypp=='Washington')and(away_team_ypp=='Oregon')").iloc[:,45:55])
#display(pointsdata.query("season_ypp==2023").query("(home_team_ypp=='Washington')and(away_team_ypp=='Oregon')").iloc[:,55:65])




,week_ypp,neutral_site_ypp,home_team_ypp,away_team_ypp,home_points_ypp,away_points_ypp,pregame_home_off_ypp,pregame_away_def_ypp
349,7,False,Washington,Oregon,36.0,33.0,1783,1612
741,14,True,Washington,Oregon,34.0,31.0,1708,1626


,pregame_away_off_ypp,pregame_home_def_ypp,pregame_home_off_TD,pregame_away_def_TD,pregame_away_off_TD,pregame_home_def_TD,pregame_home_off_TO,pregame_away_def_TO
349,1708,1590,1780.0,1627.0,1689.0,1605.0,1798,1653
741,1789,1568,1731.0,1700.0,1799.0,1582.0,1683,1634


,pregame_away_off_TO,pregame_home_def_TO,pregame_home_off_MC,pregame_away_def_MC,pregame_away_off_MC,pregame_home_def_MC,pregame_home_off_FC,pregame_away_def_FC
349,1742,1585,1779,1611,1689,1612,1796,1592
741,1837,1574,1719,1650,1793,1581,1742,1672


,pregame_away_off_FC,pregame_home_def_FC,pregame_home_off_HV,pregame_away_def_HV,pregame_away_off_HV,pregame_home_def_HV,pregame_home_off_PA,pregame_away_def_PA
349,1707,1573,1786,1600,1729,1537,1774,1621
741,1836,1581,1752,1610,1824,1537,1691,1631


,pregame_away_off_PA,pregame_home_def_PA,pregame_home_off_RU,pregame_away_def_RU,pregame_away_off_RU,pregame_home_def_RU,pregame_home_off_FGA,pregame_away_def_FGA,pregame_away_off_FGA,pregame_home_def_FGA
349,1693,1599,1758,1599,1702,1588,1767.0,1610.0,1695.0,1613.0
741,1769,1578,1678,1622,1775,1566,1692.0,1627.0,1767.0,1594.0


,pregame_home_off_FG,pregame_away_def_FG,pregame_away_off_FG,pregame_home_def_FG,pregame_home_off_FD,pregame_away_def_FD,pregame_away_off_FD,pregame_home_def_FD,margin,HFA
349,1769.0,1614.0,1707.0,1604.0,1787,1601,1703,1568,3.0,1
741,1698.0,1623.0,1762.0,1603.0,1715,1620,1792,1553,3.0,0


In [335]:
rushing = pd.read_pickle("models/yardsPerRushAttempt_winp.pickle")
passing = pd.read_pickle("models/yardsPerPass_winp.pickle")
ypps = pd.read_pickle("models/ypp_winp.pickle")

In [336]:
ypps.predict([1,5])

array([0.37097083])

In [357]:
#pd.DataFrame([[1,2,3],[1,4,5]]).T

#create a list for rushing ypp
testimpact = pd.DataFrame()

for pp in [.3,.4,.5,.6,.7]:
    for p in [5,6,7,8,9]:
        testimpact = pd.concat([testimpact,pd.DataFrame([3,4,5,6], columns=['rushing']).assign(passing = p).assign(passperc = pp)])

testimpact

,rushing,passing,passperc
0,3,5,0.3
1,4,5,0.3
2,5,5,0.3
3,6,5,0.3
0,3,6,0.3
...,...,...,...
3,6,8,0.7
0,3,9,0.7
1,4,9,0.7
2,5,9,0.7


In [373]:
testimpact = testimpact.assign(ypp = lambda x: x['passing']*x['passperc']+x['rushing']*(1-x['passperc'])).assign(const = 1)

testimpact['yppwin'] = ypps.predict(testimpact[['const','ypp']])
testimpact['rushwin'] = rushing.predict(testimpact[['const','rushing','ypp']])
testimpact['passwin'] = passing.predict(testimpact[['const','passing','ypp']])

In [378]:
testimpact.assign(rushhigher = lambda x: x['rushwin']>x['passwin'])\
.query("rushhigher==True")

,rushing,passing,passperc,ypp,const,yppwin,rushwin,passwin,rushhigher
0,3,5,0.3,3.6,1,0.152851,0.170250,0.164246,True
1,4,5,0.3,4.3,1,0.245968,0.280978,0.233140,True
2,5,5,0.3,5.0,1,0.370971,0.426688,0.319872,True
3,6,5,0.3,5.7,1,0.516026,0.586342,0.421150,True
1,4,6,0.3,4.6,1,0.295995,0.323636,0.296940,True
...,...,...,...,...,...,...,...,...,...
3,6,7,0.7,6.7,1,0.713019,0.735725,0.643083,True
2,5,8,0.7,7.1,1,0.777037,0.754384,0.727079,True
3,6,8,0.7,7.4,1,0.817913,0.817032,0.762573,True
2,5,9,0.7,7.8,1,0.863027,0.831267,0.826055,True


In [398]:
testimpact.assign(rushhigher = lambda x: x['rushwin']>x['passwin'])\
.query("rushwin>=.5")\
[['rushing','rushwin','passing','passwin','passperc','ypp','yppwin']].describe()
#.query("passwin<yppwin")[['rushing','rushwin','passing','passwin','passperc','ypp','yppwin']].describe()
#.query("rushhigher==False")[['rushing','rushwin','passing','passwin','passperc']].describe()

,rushing,rushwin,passing,passwin,passperc,ypp,yppwin
count,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000
mean,5.105263,0.658536,7.543860,0.615091,0.529825,6.442105,0.654927
std,0.976221,0.100977,1.296564,0.126808,0.137536,0.670274,0.117481
min,3.000000,0.509038,5.000000,0.361844,0.300000,5.300000,0.431861
25%,5.000000,0.569880,7.000000,0.506951,0.400000,6.000000,0.578816
50%,5.000000,0.634452,8.000000,0.613989,0.500000,6.400000,0.658430
75%,6.000000,0.735725,9.000000,0.714537,0.600000,6.900000,0.746359
max,6.000000,0.877490,9.000000,0.851310,0.700000,8.100000,0.890363


In [212]:
#const,passing,ypp
passing.predict([1,10,6])

array([0.64055131])

In [211]:
#const,rushing,ypp
rushing.predict([1,6,6])

array([0.63445217])

In [214]:
passing.predict([1,6,10])

array([0.92441282])

In [215]:
rushing.predict([1,6,10])

array([0.96272259])

In [225]:
#Experiment with Rushing and Passing
rushypp = 4
passypp = 7
passtorush = .6
ypptotal = passypp*passtorush + rushypp*(1-passtorush)

In [226]:
ypptotal

5.800000000000001